import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when

import findspark
findspark.init()

create local SparkSession

In [2]:
%%time
spark = SparkSession.builder.appName('myApp').master('local[*]').getOrCreate()

23/04/22 19:10:01 WARN Utils: Your hostname, ToniPaltus resolves to a loopback address: 127.0.1.1; using 192.168.0.105 instead (on interface enp3s0)
23/04/22 19:10:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/22 19:10:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


CPU times: user 36.8 ms, sys: 52.2 ms, total: 89 ms
Wall time: 2.25 s


read csv with inferschema

In [3]:
%%time
df = spark.read.options(header=True, inferSchema=True).csv('dataset/ds_salaries.csv')

CPU times: user 3.93 ms, sys: 0 ns, total: 3.93 ms
Wall time: 2.95 s


read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [4]:
%%time
df = spark.read.options(header=True, inferSchema=True).csv('dataset/ds_salaries.csv')

CPU times: user 3.27 ms, sys: 0 ns, total: 3.27 ms
Wall time: 227 ms


write schema of scv on screen

In [5]:
%%time
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)

CPU times: user 1.42 ms, sys: 0 ns, total: 1.42 ms
Wall time: 1.72 ms


create schema of this scv

In [6]:
my_schema = StructType([
        StructField('id', IntegerType(), False),
        StructField('work_year', IntegerType(), False),
        StructField('experience_lev', StringType(), False),
        StructField('employee_type', StringType(), False),
        StructField('job_title', StringType(), False),
        StructField('salary', IntegerType(), False),
        StructField('salary_currency', StringType(), False),
        StructField('salary_in_usd', IntegerType(), False),
        StructField('employee_residence', StringType(), False),
        StructField('remote_ratio', IntegerType(), False),
        StructField('company_location', StringType(), False),
        StructField('company_size', StringType(), False)
     ])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [7]:
%%time
new_df = spark.read.options(header=True).schema(my_schema).csv('dataset/ds_salaries.csv')

CPU times: user 3.62 ms, sys: 0 ns, total: 3.62 ms
Wall time: 39.6 ms


this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [8]:
%%time
new_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_lev: string (nullable = true)
 |-- employee_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)

CPU times: user 882 µs, sys: 606 µs, total: 1.49 ms
Wall time: 1.04 ms


now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [9]:
%%time
df = df.withColumnRenamed('_c0', 'id')
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
| id|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

23/04/22 19:10:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: _c0, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: _c0 but found: 
CSV file: file:///home/tonipaltus/Innowise/spark_demo/spark_demo_course/1_PySpark_Basics/dataset/ds_salaries.csv


print data in dataframe using display(df.toPandas())

In [10]:
%%time
display(df.toPandas())

23/04/22 19:10:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: _c0, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: _c0 but found: 
CSV file: file:///home/tonipaltus/Innowise/spark_demo/spark_demo_course/1_PySpark_Basics/dataset/ds_salaries.csv


,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


CPU times: user 200 ms, sys: 4.24 ms, total: 204 ms
Wall time: 358 ms


create df_job_title that consists from all job_titles without duplicates

In [11]:
%%time
df_job_title = df.select('job_title').dropDuplicates(['job_title'])

CPU times: user 0 ns, sys: 2.67 ms, total: 2.67 ms
Wall time: 15.3 ms


print all rows from df_job_titles without truncating jobs

In [12]:
%%time
df_job_title.show(truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
+----------

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [13]:
%%time
df_analytic = df.groupby('job_title') \
        .agg(
        max('salary').alias('max_salary'),
        avg('salary').alias('avg_salary'),
        min('salary').alias('min_salary')
)

CPU times: user 3.6 ms, sys: 1.66 ms, total: 5.25 ms
Wall time: 43.9 ms


print all rows from df_analytic without trancating jobs

In [14]:
%%time
df_analytic.show(truncate=False)

+----------------------------------------+----------+------------------+----------+
|job_title                               |max_salary|avg_salary        |min_salary|
+----------------------------------------+----------+------------------+----------+
|3D Computer Vision Researcher           |400000    |400000.0          |400000    |
|Lead Data Engineer                      |276000    |140333.33333333334|56000     |
|Head of Machine Learning                |6000000   |6000000.0         |6000000   |
|Data Specialist                         |165000    |165000.0          |165000    |
|Data Analytics Lead                     |405000    |405000.0          |405000    |
|Machine Learning Scientist              |260000    |158412.5          |12000     |
|Lead Data Analyst                       |1450000   |569000.0          |87000     |
|Data Engineering Manager                |174000    |119799.8          |51999     |
|Staff Data Scientist                    |105000    |105000.0          |1050

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [15]:
%%time
windowSpec = Window.orderBy('avg_salary')
df_analytic = df_analytic.withColumn("row_id", row_number().over(windowSpec)).sort('row_id', ascending=False)

CPU times: user 1.86 ms, sys: 1.55 ms, total: 3.4 ms
Wall time: 37.1 ms


print all data from df_analytic

In [16]:
%%time
df_analytic.show(truncate=False)

23/04/22 19:10:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/22 19:10:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/22 19:10:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/22 19:10:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/22 19:10:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/22 19:10:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/22 1

+-----------------------------+----------+------------------+----------+------+
|job_title                    |max_salary|avg_salary        |min_salary|row_id|
+-----------------------------+----------+------------------+----------+------+
|Head of Machine Learning     |6000000   |6000000.0         |6000000   |50    |
|ML Engineer                  |8500000   |2676666.6666666665|14000     |49    |
|BI Data Analyst              |11000000  |1902045.3333333333|9272      |48    |
|Lead Data Scientist          |3000000   |1101666.6666666667|115000    |47    |
|Data Science Manager         |7000000   |1062598.5833333333|137141    |46    |
|Lead Data Analyst            |1450000   |569000.0          |87000     |45    |
|Data Scientist               |30400000  |508347.2027972028 |4000      |44    |
|Big Data Engineer            |1672000   |455000.0          |18000     |43    |
|Data Analytics Lead          |405000    |405000.0          |405000    |42    |
|3D Computer Vision Researcher|400000   

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [17]:
%%time
df_analytic = df_analytic.select('row_id', 'job_title', 'max_salary', 'avg_salary', 'min_salary')

CPU times: user 3.12 ms, sys: 0 ns, total: 3.12 ms
Wall time: 14.4 ms


print df_analytic now

In [18]:
%%time
df_analytic.show(truncate=False)

23/04/22 19:10:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/22 19:10:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/22 19:10:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/22 19:10:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/22 19:10:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/22 19:10:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/22 1

+------+-----------------------------+----------+------------------+----------+
|row_id|job_title                    |max_salary|avg_salary        |min_salary|
+------+-----------------------------+----------+------------------+----------+
|50    |Head of Machine Learning     |6000000   |6000000.0         |6000000   |
|49    |ML Engineer                  |8500000   |2676666.6666666665|14000     |
|48    |BI Data Analyst              |11000000  |1902045.3333333333|9272      |
|47    |Lead Data Scientist          |3000000   |1101666.6666666667|115000    |
|46    |Data Science Manager         |7000000   |1062598.5833333333|137141    |
|45    |Lead Data Analyst            |1450000   |569000.0          |87000     |
|44    |Data Scientist               |30400000  |508347.2027972028 |4000      |
|43    |Big Data Engineer            |1672000   |455000.0          |18000     |
|42    |Data Analytics Lead          |405000    |405000.0          |405000    |
|41    |3D Computer Vision Researcher|40

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [19]:
%%time
group_data = df.groupBy('experience_level').max('salary_in_usd').show()

+----------------+------------------+
|experience_level|max(salary_in_usd)|
+----------------+------------------+
|              EX|            600000|
|              MI|            450000|
|              EN|            250000|
|              SE|            412000|
+----------------+------------------+

CPU times: user 3.63 ms, sys: 48 µs, total: 3.68 ms
Wall time: 237 ms


In [20]:
%%time
# Не знаю как сделать красивее
df_exp_lvl = df.filter(
        (df['experience_level'] == 'EX') & (df['salary_in_usd'] == 600000) |
        (df['experience_level'] == 'MI') & (df['salary_in_usd'] == 450000) |
        (df['experience_level'] == 'EN') & (df['salary_in_usd'] == 250000) |
        (df['experience_level'] == 'SE') & (df['salary_in_usd'] == 412000)
        )

CPU times: user 2.8 ms, sys: 0 ns, total: 2.8 ms
Wall time: 10.5 ms


print here df_exp_lvl

In [21]:
df_exp_lvl.show(truncate=False)

+---+---------+----------------+---------------+-------------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|id |work_year|experience_level|employment_type|job_title                |salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+-------------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|33 |2020     |MI              |FT             |Research Scientist       |450000|USD            |450000       |US                |0           |US              |M           |
|37 |2020     |EN              |FT             |Machine Learning Engineer|250000|USD            |250000       |US                |50          |US              |L           |
|63 |2020     |SE              |FT             |Data Scientist           |412000|USD            |412000       |US                |

23/04/22 19:10:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: _c0, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: _c0 but found: 
CSV file: file:///home/tonipaltus/Innowise/spark_demo/spark_demo_course/1_PySpark_Basics/dataset/ds_salaries.csv


create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [22]:
%%time
df_best = df_exp_lvl.select(['id', 'experience_level', 'salary_in_usd', 'employee_residence'])

CPU times: user 2.47 ms, sys: 0 ns, total: 2.47 ms
Wall time: 8.58 ms


print df_best

In [23]:
%%time
df_best.show(truncate=False)

+---+----------------+-------------+------------------+
|id |experience_level|salary_in_usd|employee_residence|
+---+----------------+-------------+------------------+
|33 |MI              |450000       |US                |
|37 |EN              |250000       |US                |
|63 |SE              |412000       |US                |
|97 |MI              |450000       |US                |
|252|EX              |600000       |US                |
+---+----------------+-------------+------------------+

CPU times: user 2.42 ms, sys: 0 ns, total: 2.42 ms
Wall time: 82.7 ms


23/04/22 19:10:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , experience_level, salary_in_usd, employee_residence
 Schema: _c0, experience_level, salary_in_usd, employee_residence
Expected: _c0 but found: 
CSV file: file:///home/tonipaltus/Innowise/spark_demo/spark_demo_course/1_PySpark_Basics/dataset/ds_salaries.csv


drop duplicates if exist by experience_level

In [24]:
%%time
df_best = df_best.dropDuplicates(['experience_level'])

CPU times: user 1.47 ms, sys: 0 ns, total: 1.47 ms
Wall time: 2.97 ms


print df_best

In [25]:
%%time
df_best.show(truncate=False)

23/04/22 19:10:09 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , experience_level, salary_in_usd, employee_residence
 Schema: _c0, experience_level, salary_in_usd, employee_residence
Expected: _c0 but found: 
CSV file: file:///home/tonipaltus/Innowise/spark_demo/spark_demo_course/1_PySpark_Basics/dataset/ds_salaries.csv


+---+----------------+-------------+------------------+
|id |experience_level|salary_in_usd|employee_residence|
+---+----------------+-------------+------------------+
|37 |EN              |250000       |US                |
|252|EX              |600000       |US                |
|33 |MI              |450000       |US                |
|63 |SE              |412000       |US                |
+---+----------------+-------------+------------------+

CPU times: user 4.39 ms, sys: 486 µs, total: 4.88 ms
Wall time: 302 ms


create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [26]:
%%time
df_new_best = df_best.withColumn('experience_level',
                                 when(df.experience_level == 'MI', 'middle').
                                 when(df.experience_level == 'SE', 'senior').
                                 otherwise(None)
                                 ).drop('id')

CPU times: user 2.09 ms, sys: 1.62 ms, total: 3.71 ms
Wall time: 25.3 ms


print df_new_best

In [27]:
%%time
df_new_best.show(truncate=False)

+----------------+-------------+------------------+
|experience_level|salary_in_usd|employee_residence|
+----------------+-------------+------------------+
|null            |250000       |US                |
|null            |600000       |US                |
|middle          |450000       |US                |
|senior          |412000       |US                |
+----------------+-------------+------------------+

CPU times: user 1.31 ms, sys: 1.01 ms, total: 2.32 ms
Wall time: 243 ms


write df_new_best like 1.csv and load then it to df_final

In [28]:
%%time
df_new_best.write.options(header=True).format('csv').mode('overwrite').save('dataset/1.csv')
df_final = spark.read.options(header=True).format('csv').load('dataset/1.csv')

CPU times: user 1.27 ms, sys: 983 µs, total: 2.26 ms
Wall time: 394 ms


print df_final

In [29]:
%%time
df_final.show(truncate=False)

+----------------+-------------+------------------+
|experience_level|salary_in_usd|employee_residence|
+----------------+-------------+------------------+
|null            |250000       |US                |
|null            |600000       |US                |
|middle          |450000       |US                |
|senior          |412000       |US                |
+----------------+-------------+------------------+

CPU times: user 0 ns, sys: 1.46 ms, total: 1.46 ms
Wall time: 60.1 ms


filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [30]:
%%time
df_final = df_final.dropna(subset='experience_level')
df_final.join(df, (df_final.salary_in_usd == df.salary_in_usd) & (df_final.employee_residence == df.employee_residence), 'inner').show(truncate=False)

+----------------+-------------+------------------+---+---------+----------------+---------------+----------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|experience_level|salary_in_usd|employee_residence|id |work_year|experience_level|employment_type|job_title             |salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+----------------+-------------+------------------+---+---------+----------------+---------------+----------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|middle          |450000       |US                |33 |2020     |MI              |FT             |Research Scientist    |450000|USD            |450000       |US                |0           |US              |M           |
|senior          |412000       |US                |63 |2020     |SE              |FT             |Data Scientist    

23/04/22 19:10:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: _c0, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: _c0 but found: 
CSV file: file:///home/tonipaltus/Innowise/spark_demo/spark_demo_course/1_PySpark_Basics/dataset/ds_salaries.csv


print df_final

In [31]:
df_final.show()

+----------------+-------------+------------------+
|experience_level|salary_in_usd|employee_residence|
+----------------+-------------+------------------+
|          middle|       450000|                US|
|          senior|       412000|                US|
+----------------+-------------+------------------+



last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [32]:
%%time
biggest_salary = df_final.agg(max('salary_in_usd')).collect()[0][0]
print(biggest_salary)

450000
CPU times: user 3.1 ms, sys: 1.06 ms, total: 4.16 ms
Wall time: 138 ms


It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system